# Comm 13

We need to supply electricity to a remote region that is not part of our network. A small gas generator will be ideal for this environment, but we are excited to try accompanying it with a new 80 MWh battery. This battery will allow us to have days where we do not need to turn on the generator, a positive outcome for the area.

The forecast daily electricity demands (MWh) for June are as follows:

31, 32, 39, 32, 36, 25, 33, 33, 34, 36, 36, 28, 28, 47, 33, 40, 41, 27, 37, 29, 29, 42, 49, 36, 38, 32, 29, 32, 27, 42

Each night we put in our order for the gas to power the generator for the following day. Effectively, our orders are placed in integer numbers of MWh, $x$, with a total cost of buying the gas and running the generator for a day of $\$300+80x^{0.9}$.

We need to meet the forecast demand each day, either directly from the generator or from the battery. Any surplus electricity will be stored in the battery. Once the battery is at capacity, subsequent gas is wasted.

Suppose our battery is initially empty and can be left empty at the end of the month. How much electricity should we plan to generate for each day in June? \
**[comm13.csv](https://drive.google.com/file/d/1aSbNZiXfF_SQvbSkLiN9NBIIfrsZSDws/view?usp=sharing)**

Please provide us with the optimal total cost.\
**\$55329.57**


## Formulas
Sets and Data: \
$t \quad$ Days of June $1,2,\cdots,30$ \
$\text{order}_t \quad$ electricity to generate for each day \
$\text{battery}_t \quad$ remaining electricity in the battery for each day \
$\text{demand}_t \quad$ electricity demands (MWh) for each day \
$\text{batteryCapacity} \quad$ 80 (MWh)

Transformation functions:
$$ \text{leftOver}_{\text{order}} = \text{battery}_t + \text{order}_t - \text{demand}_t $$
$$ f(x) = 
  \begin{cases}
    0, & \text{if } x=0 \\
    300 + 80x^{0.9}, & \text{if } x>0 \\
  \end{cases}
$$
$$ \text{cost}_t = f(\text{order}_t) $$

Value function:
$$ V_t(s_t) = \min \{ \text{cost}(\text{order}_t) + v_{t+1}(\text{leftOver}_{\text{order}}) \} $$

Constraints:
$$ 0 \leq \text{battery}_t + \text{order}_t - \text{demand}_t \leq 80 $$

Stopping condition: \
$V_{31}(\text{battery}_{31}) = 0$ \

Target: \
$V_1(0)$


## Solution

In [ ]:
import pandas as pd

# Data % Sets
demand = [31, 32, 39, 32, 36, 25, 33, 33, 34, 36, 36, 28, 28, 47, 33, 40, 41, 27, 37, 29, 29, 42, 49, 36, 38, 32, 29, 32, 27, 42]
batteryCapacity = 80
num_days = len(demand)

# Cost function
def cost(x):
    if x == 0:
        return 0
    else:
        return 300 + 80*x**(0.9)

# Optimizing function
_minCost = {}
def V(t,battery):
    if t == num_days:
        return (0, None)
    else:
        leftOver ={}
        for order in range(batteryCapacity + demand[t] + 1):
            leftOver[order] = battery + order - demand[t]

            
        if (t, battery) not in _minCost:
            _minCost[(t,battery)] = min((cost(order) + V(t+1,leftOver[order])[0]
                                         , order, leftOver[order])
                              for order in range(batteryCapacity + demand[t] + 1) if leftOver[order] >= 0 and 
                              leftOver[order] <= batteryCapacity)
            
        return _minCost[(t, battery)]
target = V(0,0)

# Printings & data output
days = []
production = []
remaining = []
battery = 0
for t in range(num_days):
    (cost, order, save) = V(t, battery)
    battery = save
    days.append(t+1)
    production.append(order)
    remaining.append(save)
df1 = pd.DataFrame(list(zip(days, production, remaining)), columns =['Day', 'Production', 'Saving'])
df1.to_csv('comm12.csv', index=False)
print(df1)   
print(target)    


# Comm 14

It turns out that our initial forecasts for June were rather limited. Each day there is in fact a 40% chance that demand will be higher than normal, as follows:

54, 57, 65, 54, 59, 45, 58, 54, 52, 66, 55, 44, 49, 83, 59, 66, 68, 49, 62, 54, 53, 79, 82, 61, 65, 53, 47, 51, 50, 65

The remaining 60% of the time the demand will be as previously forecast. Unfortunately, we do not know whether demand will be high or normal when we order the gas the night before.

Please provide us with the optimal expected total cost. \
**\$73593.87**

## Formulas

Data and Sets: \
$ t \in T \quad $ days of Junes $\{1, \cdots, 30\}$ \
$ a_t \quad $ electricity to generate for each day (order) \
$ s_t \quad $ remaining electricity in the battery for each day \
$ \text{batteryCapacity} = 80 $ \
$ \text{demandNormal} \quad $ normal demand \
$ \text{demandHigh} \quad $ high demand \
$ \text{HighChance} \quad $ 40% chance for high demand \


Transformation function: 
$$ \text{cost}(x) = 
  \begin{cases}
    0, & \text{if } x=0 \\
    300 + 80x^{0.9}, & \text{if } x>0 \\
  \end{cases} $$

$$ V_t(s_t) = \min_{a_t} \{ (1-\text{HighChance}) (\text{cost}(a_t) + v_{t+1}(s_t+a_t-\text{demandNormal}_t)) + \text{HighChance} (\text{cost}(a_t) + v_{t+1}(s_t+a_t-\text{demandHigh}_t)) \} $$

Constraints:
$$ s_t+a_t-\text{demandHigh}_t \geq 0 $$
$$ s_t+a_t-\text{demandHigh}_t \leq \text{batteryCapacity} $$

$$ s_t = 
  \begin{cases}
    s_t, & \text{if } s_t \leq \text{batteryCapacity} \\
    \text{batteryCapacity}, & \text{if } s_t > \text{batteryCapacity} \\
  \end{cases} $$

Stopping condition: \
$ V_{31}(s_{31}) = 0 $ \

Target: \
$ V_1(0) $


## Solution

In [ ]:
import math
import pandas as pd

# Data % Sets
demand_normal = [31, 32, 39, 32, 36, 25, 33, 33, 34, 36, 36, 28, 28, 47, 33, 40, 41, 27, 37, 29, 29, 42, 49, 36, 38, 32, 29, 32, 27, 42]
demand_high = [54, 57, 65, 54, 59, 45, 58, 54, 52, 66, 55, 44, 49, 83, 59, 66, 68, 49, 62, 54, 53, 79, 82, 61, 65, 53, 47, 51, 50, 65]
batteryCapacity = 80
num_days = len(demand_normal)

# Cost function
def cost(x):
    if x == 0:
        return 0
    else:
        return 300 + 80*x**(0.9)

# Optimizing function
_minCost = {}
def V(t,s):
    if t == num_days:
        return (0, None)
    else:
        if (t,s) not in _minCost:
            _minCost[(t,s)] = min(((0.6*(cost(a) + V(t+1,s+a-demand_normal[t])[0]) + 
                                   0.4*(cost(a) + V(t+1,s+a-demand_high[t])[0])), a)
                              for a in range(math.ceil(batteryCapacity + demand_high[t] + 1))
                              if s+a-demand_high[t] >= 0 and 
                              s+a-demand_normal[t] <= batteryCapacity)       
        return _minCost[(t,s)]

target = V(0,0) 
print(target)    

# Comm 15

In line with the demand-reduction strategy you helped us with in Communication 12, for this new region in June we have been allocated 5 days for which we can reduce the chance of high demand from 40% to only 10% (with a 90% chance of normal demand instead). We can make this request the night before, around the same time that we order our gas. The days do not have to be consecutive. 

Given this opportunity, please provide us with the optimal expected total cost. \
**\$70901.46**

## Formulas

New state: \
$n \quad$ days of 10% chance for high demand

$ k =
\begin{cases}
1, &\text{if reduce the chance of high demand to 10%} \\
0, &\text{if not} \\
\end{cases}
$

## Solution

In [ ]:
import math
import pandas as pd

# Data % Sets
demand_normal = [31, 32, 39, 32, 36, 25, 33, 33, 34, 36, 36, 28, 28, 47, 33, 40, 41, 27, 37, 29, 29, 42, 49, 36, 38, 32, 29, 32, 27, 42]
demand_high = [54, 57, 65, 54, 59, 45, 58, 54, 52, 66, 55, 44, 49, 83, 59, 66, 68, 49, 62, 54, 53, 79, 82, 61, 65, 53, 47, 51, 50, 65]
batteryCapacity = 80
num_days = len(demand_normal)
highChance = [0.4, 0.1]

# Cost function
def cost(x):
    if x == 0:
        return 0
    else:
        return 300 + 80*x**(0.9)

# Optimizing function
_minCost = {}
def V(t,s,n):
    if t == num_days:
        return (0, None)
    else:
        if s > batteryCapacity:
            s = batteryCapacity
        if (t,s,n) not in _minCost:
            _minCost[(t,s,n)] = min((((1-highChance[k])*(cost(a) + V(t+1,s+a-demand_normal[t],n-min(n,k))[0]) + 
                                   highChance[k]*(cost(a) + V(t+1,s+a-demand_high[t],n-min(n,k))[0])), a)
                              for k in range(2) for a in range(math.ceil(batteryCapacity + demand_high[t] + 1))
                              if s+a-demand_high[t] >= 0 and 
                              s+a-demand_high[t] <= batteryCapacity
                              and n-k >= 0)
        return _minCost[(t,s,n)]

target = V(0,0,5) 
print(target) 

In [ ]:
batteryCapacity = 80
highChance = [0.4, 0.1]
num_days = len(demand_normal)

# Cost function
def cost(x):
    if x == 0:
        return 0
    else:
        return 300 + 80*x**(0.9)


def expectedCost(s, k, order, t, n):
    return (1-highChance[k])*(cost(order) + V(t+1,s+ order - demand_normal[t] ,n - min(n, k))[0]) + \
            highChance[k] *(cost(order) + V(t+1,s+order -demand_high[t], n - min(n, k))[0])
    
# Optimizing function
_minCost = {}
def V(t,s,n):
    if t == 30:
        return (0, None)
    else:
        if s > batteryCapacity:
            s = batteryCapacity
            
        if (t,s,n) not in _minCost:
            _minCost[(t,s,n)] = min(((expectedCost(s,k,order,t, n)), order , s+order -demand_normal[t], k) 
                 for k in range(2) for order in range(80 + demand_high[t] + 1) 
                 if s+order -demand_high[t] >= 0 and s+order -demand_high[t] <= 80 and n-k >= 0)
        
        return _minCost[(t,s,n)]

target = V(0,0,5) 
print(target)    

battery = 0
previous = 0
n = 5
for t in range(30):
    (cost, order, save, k) = V(t, battery, n)
    print(t, "Cost:", cost, "Order:", order, "Save:", save, "yes/no", k, "left",n)
    n = n - k
    battery = save


# Comm 16

Our forecasters have advised us that high and normal demands tend to come in runs, so if demand is high one day, then there is actually a 50% chance it will be high again the next day, while if it is normal one day then there is only a 20% chance it will be high the next day. Requesting the next day for demand reduction will still reduce this to 10%, regardless of whether it was high or normal the previous day.

Based on this updated forecasting, please provide us with the optimal expected total cost. You can assume May 31st has normal demand. \
**\$65601.65**

## Formulas

$ p =
\begin{cases}
0, &\text{if normal demand for previous day} \\
1, &\text{if high demand for previous day} \\
\end{cases}
$

$ \text{highChance}_{kp} =
\begin{cases}
0.2, &\text{if } k=0, p=0 \\
0.5, &\text{if } k=0, p=1 \\
0.1, &\text{if } k=1 \\
\end{cases}
$

## Solution

In [ ]:

# Cost function
def cost(x):
    if x == 0:
        return 0
    else:
        return 300 + 80*x**(0.9)


def expectedCost(s, k, order, t, n, previous):
    if previous == 1:
        highchance = 0.5
    elif previous == 0:
        highchance = 0.2
    
    if k == 1:
        highchance = 0.1
    
    return (1-highchance)*(cost(order) + V(t+1,s+ order - demand_normal[t] ,n - min(n, k), 0)[0]) + \
            highchance *(cost(order) + V(t+1,s+order -demand_high[t], n - min(n, k), 1)[0])
    
# Optimizing function
_minCost = {}
def V(t,s,n,previous):
    if t == 30:
        return (0, None)
    else:
        if s > batteryCapacity:
            s = batteryCapacity
            
        if (t,s,n) not in _minCost:
            _minCost[(t,s,n)] = min(((expectedCost(s,k,order,t, n, previous)), order , s+order -demand_normal[t], k) 
                 for k in range(2) for order in range(80 + demand_high[t] + 1) 
                 if s+order -demand_high[t] >= 0 and s+order -demand_high[t] <= 80 and n-k >= 0)
        
        return _minCost[(t,s,n)]

target = V(0,0,5,0) 
print(target)    

battery = 0
previous = 0
n = 5
for t in range(30):
    (cost, order, save, k) = V(t, battery, n, previous)
    print(t, "Cost:", cost, "Order:", order, "Save:", save, "yes/no", k, "left",n)
    n = n - k
    battery = save


# Boss Report

* Latex: https://www.overleaf.com/project/60593c97d799975d30008d75

# Client Presentation



*   PPT: https://docs.google.com/presentation/d/1pSzVtPZVERLpk4vslnR1wnQeQqnBYmLStETp6dO0fhE/edit?usp=sharing
*   PPT Template: https://drive.google.com/file/d/10_WoSpC5B4By-ZPxMM2rhkTvjKeKHUbf/view?usp=sharing


*   Draft: https://docs.google.com/document/d/1uncAf_oBJyer2BKVNtgnyaHF6B07oGE4EOY4pNq9ls0/edit?usp=sharing

